# Genentech Cervical Cancer - XGB

https://www.kaggle.com/c/cervical-cancer-screening/

In [1]:
# imports
import sys # for stderr
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# settings 
%logstop
%logstart  -o 'xgb_nb.log' rotate
plt.style.use('ggplot')
# constants
# plt.rcParams['figure.figsize'] = (10.0, 10.0)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', 50)

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : xgb_nb.log
Mode           : rotate
Output logging : True
Raw input log  : False
Timestamping   : False
State          : active


In [3]:
# versions 
import sys
print(pd.datetime.now())
print('Python: '+sys.version)
print('numpy: '+np.__version__)
print('pandas: '+pd.__version__)
print('sklearn: '+skl.__version__)

2016-04-11 13:41:12.596033
Python: 2.7.11 |Anaconda 2.4.0 (x86_64)| (default, Dec  6 2015, 18:57:58) 
[GCC 4.2.1 (Apple Inc. build 5577)]
numpy: 1.10.0
pandas: 0.17.1
sklearn: 0.18.dev0


In [4]:
import datetime
start = datetime.datetime.now()
print(start)

2016-04-11 13:41:13.187854


## Load Train

In [5]:
train_file = './features/train_new_bt_encoded.csv'
train = pd.read_csv(train_file) #, parse_dates=[7,8])

IOError: File ./features/train_new_bt_encoded.csv does not exist

In [6]:
train.set_index('patient_id', inplace=True)
train[:3]

,patient_age_group,patient_state,ethinicity,household_income,education_level,is_screener,first_visit,last_visit,date_delta,first_after_2012,...,V24.2,V25.2,V27.0,V28.3,V70.0,V74.5,have_cancer,sum_cancer,age_pct,state_pct
patient_id,,,,,,,,,,,,,,,,,,,,,
336201912,10,41,1,0,3,1,2009,2014,68,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.538345,0.556098
94237712,5,29,1,0,3,1,2008,2014,84,0,...,NaN,NaN,1,1,1,NaN,NaN,NaN,0.624289,0.608479
186124512,1,4,1,0,3,0,2009,2014,69,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.718529,0.526563


In [7]:
skip_cols = []

In [8]:
# take only a subset of the features, the rest I think is junk

cols = list(train.columns)
cols = [x for x in cols if x not in skip_cols]

test_cols = list(cols)
test_cols.remove('is_screener')

## XGBoost

In [1]:
import xgboost as xgb

In [2]:
xgb.__version__

'0.4'

In [10]:
# support class to redirect stderr
class flushfile():
    def __init__(self, f):
        self.f = f
    def __getattr__(self,name): 
        return object.__getattribute__(self.f, name)
    def write(self, x):
        self.f.write(x)
        self.f.flush()
    def flush(self):
        self.f.flush()

In [11]:
train.fillna(0,inplace=True)

In [12]:
dtrain = train[cols].copy()

In [13]:
dtrain[:5]

,patient_age_group,patient_state,ethinicity,household_income,education_level,is_screener,first_visit,last_visit,date_delta,first_after_2012,...,V24.2,V25.2,V27.0,V28.3,V70.0,V74.5,have_cancer,sum_cancer,age_pct,state_pct
patient_id,,,,,,,,,,,,,,,,,,,,,
336201912,10,41,1,0,3,1,2009,2014,68,0,...,0,0,0,0,0,0,0,0,0.538345,0.556098
94237712,5,29,1,0,3,1,2008,2014,84,0,...,0,0,1,1,1,0,0,0,0.624289,0.608479
186124512,1,4,1,0,3,0,2009,2014,69,0,...,0,0,0,0,0,0,0,0,0.718529,0.526563
767144212,2,34,1,0,3,1,2012,2012,0,1,...,1,0,1,1,1,0,0,0,0.703938,0.741726
104743512,3,11,1,0,3,0,2008,2013,57,0,...,0,0,0,0,0,0,0,0,0.677190,0.382079


In [14]:
xval  = dtrain.iloc[9::10,:]
xhold = dtrain.iloc[10::10,:]
xtrain = dtrain.loc[~dtrain.index.isin(xval.index),:]
xtrain = xtrain.loc[~xtrain.index.isin(xhold.index),:]
print(xval.shape, xhold.shape, xtrain.shape)

((115781, 73), (115781, 73), (926255, 73))


In [15]:
Y_train = xtrain.is_screener
X_train = xtrain.drop('is_screener',axis=1)
print('X:',X_train.shape, 'Y:',Y_train.shape)

('X:', (926255, 72), 'Y:', (926255,))


In [16]:
Y_val = xval.is_screener
X_val = xval.drop('is_screener',axis=1)
print(X_val.shape, Y_val.shape)

((115781, 72), (115781,))


In [17]:
Y_hold = xhold.is_screener
X_hold = xhold.drop('is_screener',axis=1)
print(X_hold.shape, Y_hold.shape)

((115781, 72), (115781,))


In [18]:
xg_train = xgb.DMatrix(X_train, label=Y_train)
xg_val = xgb.DMatrix(X_val, label=Y_val)
watchlist = [(xg_val, 'validation'), (xg_train, 'train')]

In [20]:
X_train[:2]

,patient_age_group,patient_state,ethinicity,household_income,education_level,first_visit,last_visit,date_delta,first_after_2012,visits,...,V24.2,V25.2,V27.0,V28.3,V70.0,V74.5,have_cancer,sum_cancer,age_pct,state_pct
patient_id,,,,,,,,,,,,,,,,,,,,,
336201912,10,41,1,0,3,2009,2014,68,0,37,...,0,0,0,0,0,0,0,0,0.538345,0.556098
94237712,5,29,1,0,3,2008,2014,84,0,129,...,0,0,1,1,1,0,0,0,0.624289,0.608479


In [21]:
# Stderr redirects
oldstderr = sys.stderr # global

def capture_stderr(log):
    oldstderr = sys.stderr
    sys.stderr = open(log, 'w')
    sys.stderr = flushfile(sys.stderr)
    return log
    
def restore_stderr():
    sys.stderr = oldstderr

In [22]:
def parse_xgblog(xgblog):
    import re
    pattern = re.compile(r'^\[(?P<round>\d+)\]\s*\D+:(?P<validation>\d+.\d+)\s*\D+:(?P<train>\d+.\d+)')
    xgb_list = []
    with open(xgblog, "r") as ins:
        next(ins)
        for line in ins:
            match = pattern.match(line)
            if match:
                idx = int(match.group("round"))
                validation = float(match.group("validation"))
                training = float(match.group("train"))
                xgb_list.append([idx, validation, training])
            else:
                pass # raise Exception("Failed to parse!")
    return xgb_list

In [23]:
def plot_lcurve(errlog, validation=True):
    figsize=(12.0, 8.0)
    learning_curve = pd.DataFrame(parse_xgblog(errlog))
    learning_curve.columns = ['round','validation','training   ']
    learning_curve.set_index('round',inplace=True)
    print(learning_curve.columns)
    if not validation: # ignore validation if it is in-sample
        learning_curve.drop('validation', axis=1,inplace=True)
    ax = learning_curve.plot(figsize=figsize, title='Learning Curve', legend=True)
    legends = learning_curve.columns.values
    lego = [i+' '+str(learning_curve.iloc[-1,j]) for i,j in zip(legends,range(len(legends)))]
    leg = plt.legend(lego, loc='center right')
    plt.show()
    return ax

In [24]:
# Plot of a ROC curve
def plot_auc(fpr, tpr, roc_auc, label=''):
    fig = plt.figure(figsize=(12.0, 8.0))
    ax = fig.add_subplot(111)
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    ax.plot([0, 1], [0, 1], 'k--')
    ax.plot(fpr, tpr, label=label+' AUC = %0.2f' % (roc_auc*100))
    plt.legend(loc="lower right")
    plt.show()
    return ax

In [41]:
# set params
params = {'objective': 'binary:logistic',
          'booster': 'gbtree',
          'eta': 0.5,
          'max_depth': 7,
          'colsample_bylevel':1.0,
#          'colsample_bytree': 0.3,
          'silent': 1,
          'eval_metric': 'auc',
          'seed': 8888
          }

num_round = 100
early_stopping_rounds=10

In [ ]:
# Train an XGBoost model
xgblog = capture_stderr('xgb.log')

bst = xgb.train(params, xg_train, num_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, 
                 verbose_eval=True)

restore_stderr()

In [ ]:
ax = plot_lcurve(xgblog)

In [ ]:
preds = bst.predict(xg_val)

In [ ]:
len(Y_val.values), len(preds)

In [ ]:
xgb_fpr, xgb_tpr, _ = metrics.roc_curve(Y_val.values, preds)
xgb_roc_auc = metrics.auc(xgb_fpr, xgb_tpr)

In [ ]:
ax = plot_auc(xgb_fpr, xgb_tpr, xgb_roc_auc, label='XGB')
#plot_auc(fpr, tpr, roc_auc, label='Mean')

In [ ]:
# Compute Precision-Recall and plot curve
def plot_precision_recall(actuals, preds):
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import average_precision_score

    precision, recall, _ = precision_recall_curve(actuals, preds)
    average_precision = average_precision_score(actuals, preds)

    # Plot Precision-Recall curve
    fig = plt.figure(figsize=(12.0, 8.0))
    plt.clf()
    plt.plot(recall, precision, label='Precision-Recall curve')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall: AUC={0:0.2f}'.format(average_precision))
    plt.legend(loc="lower left")
    plt.show()
    return

In [ ]:
plot_precision_recall(Y_val.values, preds)

In [ ]:
# Funcs from https://github.com/stripe/topmodel/

def get_thresholds_trues_totals(range_info, bin_list, predicted, actual, weight=1.0):
    trues, totals, thresholds = [], [], []
    for i in range(range_info):
        thresholds.append(bin_list[i + 1])
        obs_in_bin = (predicted >= bin_list[i]) & (predicted < bin_list[i + 1])
        true_obs_in_bin = obs_in_bin & actual
        trues.append(np.sum(weight * true_obs_in_bin))
        totals.append(np.sum(weight * obs_in_bin))

    return {'thresholds': thresholds, 'trues': trues, 'totals': totals}

def to_histogram_format(actual, predicted, resample=False):
    THRESHOLD_BINS = 100
    TOP_THRESHOLDS = [0.9, 0.95, 0.99, 0.995, 0.9975, 0.999, 0.9995, 0.9999, 0.99995, 0.999999, 0.9999999, 1]
    BIN_COUNT = 100

    bin_edges = map(lambda x: x * 1.0 / THRESHOLD_BINS, range(0, THRESHOLD_BINS + 1))
    top_bins = TOP_THRESHOLDS[:]
    top_bins.insert(0, 0.0)
    ret = get_thresholds_trues_totals(THRESHOLD_BINS, bin_edges, predicted, actual)
    if not resample:
        high_end = get_thresholds_trues_totals(len(TOP_THRESHOLDS), top_bins, predicted, actual)
        ret['high_end_hist'] = high_end

    return ret

In [ ]:
def plot_absolute_score_histogram(thresholds, all_counts, xlabel, true_counts=None, ax=None):
    figsize=(12.0, 8.0)
    # First graph
    if ax is None:
        _, ax = plt.subplots(figsize=figsize)
    width = (thresholds[1] - thresholds[0]) / 2
#    width = []
#     b = thresholds[0]
#     for i in thresholds[1:]:
#         width.append(i - b)
#         b = i
    offset = [i + width for i in thresholds]
    # offset = [i + j for i, j in zip(thresholds,width)]
    if true_counts is not None:
        falses = [i - j for i, j in zip(all_counts, true_counts)]
        plt.bar(offset, falses , width=width, log=False, label="False items")
        plt.bar(thresholds, true_counts, width=width, log=False, color="purple", label="True items")
    else:
        plt.bar(thresholds, all_counts, width=width, log=False, color="purple", label="All items")
    plt.grid(False)
    plt.xlim((thresholds[0], thresholds[-1] + 0.01))
    plt.xlabel(xlabel)
    plt.legend(loc='best')
    plt.title('Absolute Score Distribution')
    plt.show()
    return

In [ ]:
th = to_histogram_format(Y_val.values, preds)
plot_absolute_score_histogram(th['thresholds'], th['totals'], xlabel='Score', true_counts=th['trues'])

In [ ]:
# not working yet
# th2 = th['high_end_hist']
# plot_absolute_score_histogram(th2['thresholds'], th2['totals'], xlabel='High End Score', true_counts=th2['trues'])

In [ ]:
def plot_scores_histogram_log(thresholds, all_counts, xlabel, true_counts=None, ax=None):
    import matplotlib
    figsize=(12.0, 8.0)
    # First graph
    if ax is None:
        _, ax = plt.subplots(figsize=figsize)
    width = (thresholds[1] - thresholds[0]) / 2
    offset = [i + width for i in thresholds]
    if true_counts is not None:
        falses = [i - j for i, j in zip(all_counts, true_counts)]
        plt.bar(offset, falses, width=width, log=True, label="False items")
        plt.bar(thresholds, true_counts, width=width, log=True, color="purple", label="True items")
    else:
        plt.bar(thresholds, all_counts, width=width, log=True, color="purple", label="All items")
    plt.grid(False)
    ax.yaxis.set_major_formatter(matplotlib.ticker.ScalarFormatter())
    ax.yaxis.get_major_formatter().set_scientific(False)
    plt.xlim((0.0, 1.01))
    plt.xlabel(xlabel)
    plt.legend(loc='best')
    plt.title('Log Score Distribution')
    plt.show()
    return

In [ ]:
plot_scores_histogram_log(th['thresholds'], th['totals'], xlabel='Score', true_counts=th['trues'])

In [ ]:
import gc
gc.collect()

In [ ]:
break # check that everything is good before proceeding

## Retrain against full data

In [ ]:
xtrain = train[cols].iloc[:,:] 
xval   = train[cols].iloc[9::10,:]

In [ ]:
Y_train = xtrain.is_screener
X_train = xtrain.drop('is_screener',axis=1)
Y_val = xval.is_screener
X_val = xval.drop('is_screener',axis=1)
print('Xt:',X_train.shape, 'Yt:',Y_train.shape, 
      'Xv:', X_val.shape, 'Yv:', Y_val.shape)

In [ ]:
xg_train = xgb.DMatrix(X_train, label=Y_train)
xg_val = xgb.DMatrix(X_val, label=Y_val)
watchlist = [(xg_val, 'validation'), (xg_train, 'train')]

In [ ]:
# leave params same as before

num_round = 200
early_stopping_rounds=10

In [ ]:
# Train an XGBoost model
xgbplog = capture_stderr('xgb_final.log')

bst = xgb.train(params, xg_train, num_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, 
                 verbose_eval=True)

restore_stderr()

In [ ]:
ax = plot_lcurve(xgbplog, validation=False)

## Predict Test Set

In [ ]:
test_file = './features/test_bt_encoded.csv'
test = pd.read_csv(test_file)
test.set_index('patient_id', inplace=True)
test[:3]

In [ ]:
for physician in ['GO','GYN','OBG','OBS','OCC','ON','OTO','PCP','REN']:
    test[test[physician] > 0] = 1

In [ ]:
test.fillna(0, inplace=True)
xtest = test[test_cols].copy()

In [ ]:
xtest[:5]

In [ ]:
xg_test = xgb.DMatrix(xtest)

In [ ]:
test_preds = bst.predict(xg_test)

In [ ]:
df_preds = pd.DataFrame(test_preds, index=xtest.index, columns=['predict_screener'])

In [ ]:
gc.collect()

## Compare to best LB score

In [ ]:
def compare_plot(best, current, set_aspect=False):
    '''
    '''
    best = best.sort_index()
    current = current.sort_index()
    m, b = np.polyfit(best,current, 1)
    x = np.arange(min(best),max(best))
    fig = plt.figure(figsize=(12.,8.))
    plt.scatter(best, current, marker='o', s=1)
    plt.plot(x, m*x + b, 'r-', label='%.4f'%m)
    if set_aspect:
        plt.gca().set_aspect('equal', 'box')
    plt.suptitle('Comparing Predictions')
    plt.xlabel('Best Prediction')
    plt.ylabel('New Prediction')
    plt.legend()
    print(m, b)
    plt.show()
    return

In [ ]:
best = pd.read_csv('./output/Multi-model-VW-XGB-best-II.csv.gz')

In [ ]:
test_exclude = pd.read_csv('./input/test_patients_to_exclude.csv', header=None, names=['patient_id'])

In [ ]:
best.set_index('patient_id', inplace=True)
best[:2]

In [ ]:
best.shape, df_preds.shape

In [ ]:
best.drop(test_exclude.patient_id, inplace=True)

In [ ]:
best.shape[0] - len(test_exclude.patient_id) , df_preds.shape

In [ ]:
best.shape, df_preds.shape

In [ ]:
compare_plot(best.predict_screener, df_preds.predict_screener)

## DONE !!! Submit it!

In [ ]:
sample = pd.read_csv('./output/sample_submission.csv')
sample.set_index('patient_id', inplace=True)
sample.sort_index(inplace=True)
sample[:2]

In [ ]:
df_preds.shape, sample.shape

In [ ]:
df_preds.sort_index(inplace=True)

In [ ]:
df_preds[:10]

In [ ]:
sample.update(df_preds, join='left', overwrite=True)
sample[:10]

In [ ]:
if sample.shape[0] != 2169045:
    raise Exception('Final prediction file of wrong size!')

In [ ]:
pred_file = './output/submit_xgb_bug.csv'

In [ ]:
sample.to_csv(pred_file)

In [ ]:
! gzip -f {pred_file}

In [ ]:
end   = datetime.datetime.now()
print('run time: '+str(end-start)+' at: '+str(end))